## Import All Necessary Modules And Setup Project
If you get any errors when importing these, ensure you run the command:

  $ python -m pip install -r requirements.txt

to install all necessary modules for this project. This command must be run from inside of this project directory.

It is recommended to use virtual environments for this project to ensure there is no conflicting package versions on your system.

Activate the virtual environment (if needed), run the pip install command, and then launch Jupyter Lab inside this project to get this project running.

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats
import numpy as np
from scipy.stats import t
from numpy.polynomial.polynomial import polyfit

In [ ]:
masterData=pd.read_csv("data/crime-housing-austin-2015.csv")
ZipcodeData=pd.read_csv("data/AustinZipCodes.csv")
#explore dataset
masterData.head()
#It receives an int or None (to print all the columns):
pd.set_option('display.max_columns', None)
masterData.head()

In [ ]:
#list out all the columns in a dataset
list(masterData)
#list unique values in a column "Council_District" of a dataset
masterData.Council_District.unique()
#clear rows that contains nan values
masterData=masterData.dropna()
ZipcodeData=ZipcodeData.dropna()

In [ ]:
#type of a dataset columns
masterData.dtypes
#"council district" & "zip codes" are float, lets make it as int first
masterData=masterData.astype({"Council_District":'int',"Zip_Code_Crime":'int'})
#percent sign or dollar sign attached
#dataset['colname'] = dataset['colname'].str.replace('$', '').astype('float')
masterData['Medianhouseholdincome'] = masterData['Medianhouseholdincome'].str.replace('$', '').astype('float')
masterData['Medianrent'] = masterData['Medianrent'].str.replace('$', '').astype('float')
masterData['Medianhomevalue'] = masterData['Medianhomevalue'].str.replace('$', '').astype('float')
masterData['Unemployment'] = masterData['Unemployment'].str.replace('%', '').astype('int')
masterData['Populationbelowpovertylevel'] = masterData['Populationbelowpovertylevel'].str.replace('%', '').astype('int')
masterData['Non-WhiteNon-HispanicorLatino'] = masterData['Non-WhiteNon-HispanicorLatino'].str.replace('%', '').astype('int')
masterData['HispanicorLatinoofanyrace'] = masterData['HispanicorLatinoofanyrace'].str.replace('%', '').astype('int')

In [ ]:
sub_masterData=masterData[['Key','Zip_Code_Crime','District','Clearance_Status','Highest_NIBRS_UCR_Offense_Description',
                              'Medianhouseholdincome', 'Unemployment','Populationbelowpovertylevel','Medianrent',
                              'Medianhomevalue','Non-WhiteNon-HispanicorLatino','HispanicorLatinoofanyrace']]
sub_masterData=sub_masterData.rename(columns={'Zip_Code_Crime':'Zip_Code','Highest_NIBRS_UCR_Offense_Description':'crime_type',
                                             'Clearance_Status':'CS','Medianhouseholdincome':'MhhI','Unemployment':'UE(%)',
                                             'Populationbelowpovertylevel':'Pov_lvl','Medianrent':'rent',
                                             'Medianhomevalue':'home_price','HispanicorLatinoofanyrace':'hispanic',
                                             'Non-WhiteNon-HispanicorLatino':'non_wh_non_lat'})
sub_masterData['non_native_pop']=sub_masterData.non_wh_non_lat+sub_masterData.hispanic
display(sub_masterData.head())
ZipcodeData=ZipcodeData.rename(columns={'Zip Code':'Zip_Code'})
display(ZipcodeData.head())

In [ ]:
unified_dataset=pd.merge(sub_masterData,ZipcodeData,on='Zip_Code')
display(unified_dataset)
#Convert string with comma separator and dot to float in population column values
unified_dataset['Population'] = unified_dataset['Population'].str.replace(',', '').astype(float)

In [ ]:
#groupby zip code for sub_masterdata
grp_sub_masterData = unified_dataset.groupby('Zip_Code').agg(
    {'crime_type': 'count','MhhI':'mean','UE(%)':'mean','Pov_lvl':'mean','rent':'mean','home_price':'mean',
    'non_wh_non_lat':'mean','hispanic':'mean','Population':'mean','non_native_pop':'mean'})
grp_sub_masterData=grp_sub_masterData.rename(columns={'crime_type':'crime_count'})
grp_sub_masterData=grp_sub_masterData.reset_index()

grp_sub_masterData['crimepercapita']=grp_sub_masterData['crime_count']/grp_sub_masterData['Population']
grp_sub_masterData['cr_rate_nonative']=grp_sub_masterData['crime_count']/(grp_sub_masterData['Population']*grp_sub_masterData['non_native_pop']/100)
display(grp_sub_masterData.head())

In [ ]:
#aggregate the total number of each type of crimes
crime_types = sub_masterData.groupby('crime_type').agg(
    {'crime_type': 'count'})
crime_types=crime_types.rename(columns={'crime_type':'count'})
crime_types=crime_types.reset_index()

#plot figure
plt.figure(figsize=(8,8))
plt.pie(data=crime_types, x='count',shadow=True)
plt.title('Texas Crime Results in 2015')
plt.legend(labels=crime_types['crime_type'])
plt.savefig('pi_plot_categorical_crime.jpeg')

In [ ]:
#scatterplot for crime per capita for different zip codes in texas austin
plt.figure(figsize=(8,8))
plt.scatter(grp_sub_masterData.Zip_Code,grp_sub_masterData.crimepercapita)

In [ ]:
#remove outlier in a zip code whose crimepercapita >.10: the zip code 78701 has population only 3855, hence outlier
grp_sub_masterData_scatter=grp_sub_masterData.copy()
grp_sub_masterData_scatter=grp_sub_masterData_scatter[grp_sub_masterData_scatter['crimepercapita']<0.10]
plt.figure(figsize=(8,8))
plt.scatter(grp_sub_masterData_scatter.Zip_Code,grp_sub_masterData_scatter.crimepercapita)

In [ ]:
#making sub plots for comparison
fig, ax=plt.subplots(2,figsize=(6,10))
plt.title('scatter plot for different zip codes after removing opulier')
ax[0].scatter(grp_sub_masterData.Zip_Code,grp_sub_masterData.crimepercapita)
ax[0].set_xlabel('zip codes')
ax[0].set_ylabel('all types of crime per capita')
ax[1].scatter(grp_sub_masterData_scatter.Zip_Code,grp_sub_masterData_scatter.crimepercapita)
ax[1].set_xlabel('zip codes')
ax[1].set_ylabel('all types of crime per capita')
plt.savefig('scatter plot for crime per capita in different zip codes.jpeg')

### split dataset for each crime type

In [ ]:
#sub_masterData.loc[sub_masterData['crime_type'] == 'Robbery', 'crime_type'].count()
Data_robbery=unified_dataset[unified_dataset['crime_type']=='Robbery']
Data_Burglary=unified_dataset[unified_dataset['crime_type']=='Burglary']
Data_Auto_Theft=unified_dataset[unified_dataset['crime_type']=='Auto Theft']
Data_Agg_Assault=unified_dataset[unified_dataset['crime_type']=='Agg Assault']
Data_Theft=unified_dataset[unified_dataset['crime_type']=='Theft']
Data_Murder=unified_dataset[unified_dataset['crime_type']=='Murder']

In [ ]:
Data_robbery.head()

In [ ]:
Data_robbery_grp_zip=Data_robbery.groupby('Zip_Code').agg(
    {'crime_type': 'count','Pov_lvl':'mean',
    'non_wh_non_lat':'mean','hispanic':'mean','Population':'mean','non_native_pop':'mean'})
Data_robbery_grp_zip=Data_robbery_grp_zip.reset_index()
Data_robbery_grp_zip=Data_robbery_grp_zip.rename(columns={'crime_type':'crime_robbery'})
display(Data_robbery_grp_zip.head())

## main instructions
Your analysis must include a **number of statistical methods**. You must include `Pearson correlations` (be sure to report the **`p-value`**), _scatterplots_, _averages_, _standard deviations_, and a **t-test** (or **Mann-Whitney-U test**). The specific number of analyses is left up to you, but the contribution must be significant and your project report must give detailed justification and results for each analysis.

## Look at summary statistics mean, median, and mode

In [ ]:
rent=sub_masterData.rent
homeprice=sub_masterData.home_price
print('mean: {}\nmedian: {}\nstddev: {}\nmode: {}'.format(rent.mean(), rent.median(), rent.std(), rent.mode()))
print('mean: {}\nmedian: {}\nstddev: {}\nmode: {}'.format(homeprice.mean(), homeprice.median(), homeprice.std(),
                                                          homeprice.mode()))
sub_masterData.describe()

## Show a scatterplot and regression line & Pearson R
correlation between population percentage below poverty line towards the crime commit

In [ ]:
#aggregate the total number of crimes with respect to population below poverty level
crime_POV_lvl = grp_sub_masterData.groupby('Pov_lvl').agg(
    {'crimepercapita': 'mean'})
crime_POV_lvl=crime_POV_lvl.reset_index()
crime_POV_lvl=crime_POV_lvl[crime_POV_lvl['Pov_lvl']<30]
crime_POV_lvl=crime_POV_lvl[crime_POV_lvl['Pov_lvl']!=20]
crime_POV_lvl

In [ ]:
plt.figure(figsize=(8,8))
sns.regplot('Pov_lvl', 'crimepercapita', data=crime_POV_lvl)

plt.title('correlation for population % below poverty line(<30%) to crimepercapita by zip codes')
#we removed one single outlier that has crime rate per capita of 0.5 for poverty level 20
plt.savefig('correlation for population % below poverty line to crimepercapita.jpeg')
#pearson R coefficient and probaility in confidence of statistics
display(stats.pearsonr(crime_POV_lvl.Pov_lvl, crime_POV_lvl.crimepercapita))

In [ ]:
plt.figure(figsize=(8,8))
sns.regplot('rent', 'home_price', data=sub_masterData,marker='o',scatter_kws={'s':15},color='blue',line_kws={'color':'red'})
plt.title('correlation for rent paid by people to price of house')
plt.xlabel('rent paid($)')
plt.ylabel('median price of house in texas ($)')
plt.savefig('correlation for rent paid by people to price of house.jpeg')
#pearson R coefficient and probaility in confidence of statistics
display(stats.pearsonr(sub_masterData.rent, sub_masterData.home_price))

In [ ]:
plt.figure(figsize=(8,8))
sns.regplot('Population', 'home_price', data=unified_dataset,marker='o',scatter_kws={'s':15},color='blue',line_kws={'color':'red'})
plt.title('correlation for population to price of house')
plt.xlabel('population')
plt.ylabel('median price of house in texas ($)')
plt.savefig('correlation for population to price of house.jpeg')
#pearson R coefficient and probaility in confidence of statistics
display(stats.pearsonr(unified_dataset.Population, unified_dataset.home_price))

## distribution plots

In [ ]:
plt.figure()
#histogram plot or KDE plot 
#sns.kdeplot(data = sub_masterData['MhhI'])
#sns.histplot(data = sub_masterData['MhhI'],bins=11,kde=True,legend=True)
sns.distplot(sub_masterData[sub_masterData.CS == 'N'].rent, hist=False, label='not cleared',bins=None)
sns.distplot(sub_masterData[sub_masterData.CS == 'C'].rent, hist=False, label='cleared by arrest',bins=None)
sns.distplot(sub_masterData[sub_masterData.CS == 'O'].rent, hist=False, label='cleared by exception',bins=None)
plt.xlabel('median house rent($)')
plt.legend()

## T distribution
The t test computes t-statistic, which measures the spread between means. What are the chances that we re-sampled from a population and got the same value of t? This is what the p value is important for.

In [ ]:
#sns.distplot(sub_masterData.rent, hist=False)
#sns.distplot(sub_masterData.home_price, hist=False)

#t-test for rent vs house price in texas in 2015
stats.ttest_ind(sub_masterData.rent, sub_masterData.home_price)

In [ ]:
#t-test for population % below poverty level
stats.ttest_ind(sub_masterData.Pov_lvl, sub_masterData.rent)

In [ ]:
#t-test for population and house price by places
stats.ttest_ind(unified_dataset.Population, unified_dataset.MhhI)